In [1]:
import datetime
import os
import sys
import sklearn
import numpy as np
import tensorflow as tf
from scipy.io import loadmat
from matplotlib import pyplot as plt
from data_utils import get_data
from models.resnet import ResNet18
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [2]:
train_data = loadmat('../EQun/earthb.mat')
all_data = train_data['images']
all_target = train_data['labels']

# load data
x_train, x_valid, x_test, y_train, y_valid, y_test, splits, splits_test = get_data(all_data, all_target, dataset='eq', seed=1, shape=(-1, 3, 50, 40))
# set contains training and validation
X = np.concatenate([x_train, x_valid])
y = np.concatenate([y_train, y_valid])


In [ ]:
model = ResNet18([2, 2, 2, 2], n_classes=3)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])



In [ ]:

checkpoint_save_path = "./checkpoint/Baseline.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True)

history = model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_valid, y_valid), validation_freq=1,
                    callbacks=[cp_callback])

model.summary()

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_/' + current_time + '/train'
test_log_dir = 'logs/gradient_/' + current_time + '/test'

# show   ################

# 显示训练集和验证集的acc和loss曲线
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()
plt.savefig('loss_acc.png')
plt.close()



In [ ]:

result = model.predict(x_test)
preds = tf.argmax(result, axis=1)

if tf.__version__[0] == '1':
    print('Test accuracy', sklearn.metrics.accuracy_score(y_test, preds.eval(session=tf.compat.v1.Session())))
if tf.__version__[0] == '2':
    print('Test accuracy', sklearn.metrics.accuracy_score(y_test, preds.numpy()))

In [ ]:
preds.eval

'1.15.0'

array([0, 0, 0, ..., 0, 0, 0])

In [11]:
preds.eval

<bound method Tensor.eval of <tf.Tensor 'ArgMax:0' shape=(6612,) dtype=int64>>